In [ ]:
from torchtext import data

class dataProcesser():
    def __init__(self,src,des,n):
        self.src = src
        self.des = des
        self.n = n
        self.dics = {}
        self.sepToken = '<sep> '
        self.endToken = '<sep>'
        CONTEXT = data.Field()
        ANSWER  = data.Field()
        QUESTION = data.Field()
       
        # define col: {[source data col name]:[your data col name],Field}
        fields = {'context':('Context',CONTEXT),'question':('Question',QUESTION),'supporting_facts':('Answer',ANSWER)}
        dataset = data.TabularDataset(path = src,format='json',fields=fields)
        dataset = dataset.examples[0]
        
        self.dics = []
        #len(dataset.Context)
        for i in range (0,len(dataset.Context)):
            s_id = 0
            ts = {}
            for title,sentence in dataset.Context[i]:
                # title:str sentence:list     
                ts[title] = [sentence,s_id]
                s_id = s_id + len(sentence)
            self.dics.append(ts)
        self.go(dataset)
    
    def getAnswer(self,ans,idx):
        # idx: data index
        label = []
        res = ''
        dic = self.dics[idx]
        for title, sent_id in ans:
            if title in dic:
                if sent_id < len(dic[title][0]):
                    res += dic[title][0][sent_id] + self.sepToken
                    #print('write data {} {}'.format(dic[title][1],sent_id))
                    label.append(dic[title][1]+sent_id)
        return [res,label]
    
    def getContext(self,text2DimList):
        res = ''
        for paragragh in text2DimList:
            res += self.sepToken.join(paragragh[1]) + self.sepToken
        return  res  

    def go(self,dataset):
        if self.n is -1:
            self.n = len(dataset.Context)
            print('data length is',self.n)
        pbar = pyprind.ProgBar(self.n)
        with open(self.des,'w',encoding="utf-8",newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['passage','question', 'answer','label'])
            for i in range (0,self.n):
                c =  self.getContext(dataset.Context[i]).lower()
                q =  dataset.Question[i].lower()
                a,l =  self.getAnswer(dataset.Answer[i],i)
                writer.writerow([c,q,a,l])
                pbar.update()
        print('write down')  
        